# Project Title
### Data Engineering Capstone Project
## Sebastian Baeza October - 2019
#### Project Summary

The data sources will be aggregated with Spark SQL (an instance is available by project template), and pandas will be used to analyze the data.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import pandas as pd
import os
import glob
from us_state_abbrev import state_udf, abbrev_state, abbrev_state_udf,city_code_udf,city_codes
from immigration_codes import country_udf
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

In [3]:
#Build spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [4]:
#Build SQL context object
sqlContext = SQLContext(spark)

### Step 1: Scope the Project and Gather Data

#### Scope 
This project will get data from all available sources in the project template and create dimension and fact tables to show some insights of immigration in the USA.

Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 




First, i will show some rows of the source data for the project.

#### Demographic data:

In [7]:
df_demog = pd.read_csv("us-cities-demographics.csv", sep=";", header=0)

df_demog.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### Airport data:

In [9]:
df_airport = pd.read_csv("airport-codes_csv.csv", sep=",", header=0)

df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


#### Temperature Data

In [10]:
df_temp = pd.read_csv("GlobalLandTemperaturesByState.csv", sep=",", header=0)

df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


#### Immigration data

In [11]:
df_immi = pd.read_sas("../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat")

df_immi.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4.0,2016.0,6.0,135.0,135.0,b'XXX',20612.0,NaN,NaN,NaN,...,b'U',NaN,1957.0,b'10032016',NaN,NaN,NaN,1.493846e+10,NaN,b'WT'
1,5.0,2016.0,6.0,135.0,135.0,b'XXX',20612.0,NaN,NaN,NaN,...,b'U',NaN,1966.0,b'10032016',NaN,NaN,NaN,1.746006e+10,NaN,b'WT'
2,6.0,2016.0,6.0,213.0,213.0,b'XXX',20609.0,NaN,NaN,NaN,...,b'U',NaN,1989.0,b'D/S',NaN,NaN,NaN,1.679298e+09,NaN,b'F1'
3,7.0,2016.0,6.0,213.0,213.0,b'XXX',20611.0,NaN,NaN,NaN,...,b'U',NaN,1993.0,b'D/S',NaN,NaN,NaN,1.140963e+09,NaN,b'F1'
4,16.0,2016.0,6.0,245.0,245.0,b'XXX',20632.0,NaN,NaN,NaN,...,b'U',NaN,1992.0,b'D/S',NaN,NaN,NaN,1.934535e+09,NaN,b'F1'


Let's load this data in Spark...

In [8]:
# Read in the data here
spark_demographics=spark.read.format("csv").option("header", "true").option("delimiter", ";").load("us-cities-demographics.csv")

spark_airport=spark.read.format("csv").option("header", "true").load("airport-codes_csv.csv")

spark_temperature=spark.read.format("csv").option("header", "true").load("GlobalLandTemperaturesByState.csv")

spark_sas_data=spark.read.format('com.github.saurfang.sas.spark').load("../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat")

Immigration Data by State with Origin

In [9]:
# 1.- Remove nulls values
# 2.- Convert i94res codes to country of origin
# 3.- Filter out NULLS
# 4.- Run country_udf function to show state names
# country_udf, abbrev_state_udf and city_code_udf were created with data from i94 SAS labels Descriptions file.
i94_sas_data=spark_sas_data.filter(spark_sas_data.i94addr.isNotNull())\
.filter(spark_sas_data.i94res.isNotNull())\
.filter(col("i94addr").isin(list(abbrev_state.keys())))\
.filter(col("i94port").isin(list(city_codes.keys())))\
.withColumn("origin_country",country_udf(spark_sas_data["i94res"]))\
.withColumn("dest_state_name",abbrev_state_udf(spark_sas_data["i94addr"]))\
.withColumn("i94yr",col("i94yr").cast("integer"))\
.withColumn("i94mon",col("i94mon").cast("integer"))\
.withColumn("city_port_name",city_code_udf(spark_sas_data["i94port"]))

new_i94_sas_data=i94_sas_data.select("cicid",col("i94yr").alias("year"),col("i94mon").alias("month"),\
                             "origin_country","i94port","city_port_name",col("i94addr").alias("state_code"),"dest_state_name")

In [10]:
new_i94_sas_data.show(5)

+-----+----+-----+--------------+-------+------------------+----------+---------------+
|cicid|year|month|origin_country|i94port|    city_port_name|state_code|dest_state_name|
+-----+----+-----+--------------+-------+------------------+----------+---------------+
| 46.0|2016|   12|         SPAIN|    HOU|HOUSTON           |        TX|          Texas|
| 56.0|2016|   12|    CHINA, PRC|    NEW|NEWARK/TETERBORO  |        OH|           Ohio|
| 67.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        MD|       Maryland|
| 68.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        FL|        Florida|
| 69.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        HI|         Hawaii|
+-----+----+-----+--------------+-------+------------------+----------+---------------+
only showing top 5 rows



In [13]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [9]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.